In [17]:
import os

filePath = r"C:\\Users\\冰\Desktop\\NFQA后端开发\\public\\Txt"  # 文件夹路径
# fileList:存放所有文件名（包含后缀）
fileList = os.listdir(filePath)

# 读取10个文件
# fileList = fileList[:10]

# textList:存放所有文件内容
textList = []

for file in fileList:
    with open(os.path.join(filePath, file), encoding='UTF-8')as f:
    # with open(os.path.join(filePath, file))as f:
        # with open(os.path.join(filePath, file))as f:
        # print(file)  # 文件名
        # print(file)
        textList.append(f.readlines())

# result_text:对textList的内容进行处理
result_text = []
for i in textList:
    result_text.append(''.join(i))

# titleList:存放所有文件名（去除后缀）
titleList = []
for i in fileList:
    titleList.append(i[:-4])

# dataset: 存放最终数据 格式: 标题,内容
dataset = []
for i in range(len(titleList)):
    dataset.append((titleList[i], result_text[i]))


In [18]:
fileList

['2014-2015学年管理与经济学院团学评优通知.txt',
 '2015-2016学年第一学期学生教材订书通知.txt',
 '2015年11月份精品团日活动评比结果汇总表.txt',
 '2015年9月精品团日活动评比结果汇总表.txt',
 '2015年暑期赴德国游学项目通知——国际学院.txt',
 '2015年管韵—绽放管经趣味排球赛通知.txt',
 '2015昆明V主播首季评选活动方案.txt',
 '2015第五届云南省高校文化节“高教社杯”云南省大学生知识竞赛活动的通知.txt',
 '4月社区活动.txt',
 'play with ME活动通知.txt',
 '“RunningME”通知.txt',
 '“show ME your dormitory”活动通知.txt',
 '“Show Me ·I Know”挑战赛通知.txt',
 '“我爱昆工”校园文化创意产品设计大赛通知.txt',
 '“爱校文明卫生月”活动方案.txt',
 '关于“昆工创客孵化器”首批入孵团队遴选的通知.txt',
 '关于举办“MM创新孵化基地”云南移动客户端首期校园牛人推广大赛的通知.txt',
 '关于举办昆明理工大学2015年云南省大学生乐团高雅艺术进校园演出的通知.txt',
 '关于举办昆明理工大学首届公益广告设计大赛的通知.txt',
 '关于号召全校师生为第十届中国大学生年度人物投票的通知.txt',
 '关于启动昆工创客行动暨举办2015年首届昆明理工大学学生创新创业大赛的通知.txt',
 '关于在3•5学雷锋日期间集中开展志愿服务系列活动的通知.txt',
 '关于在清明节期间集中开展“清明祭英烈共铸中华魂”主题宣传教育活动的通知.txt',
 '关于开展在校生电子商务创业情况调查的通知.txt',
 '关于开展消防安全宣传教育月活动的通知.txt',
 '关于我校网络文明志愿者注册工作及基层团委考核满意度测评工作的通知.txt',
 '关于校运动会期间停课的通知11.13.txt',
 '关于管理与经济学院团委学生会2015-2016学年呈贡校区主席、部长竞选的通知(修改版).txt',
 '关于组织学生申报2016年学生课外学术科技创新基金课题的预通知.txt',
 '关于组织开展大智慧全国大学生金融精英挑战赛的通知.txt',
 '关

In [19]:
from py2neo import Graph, Node, Relationship

graph = Graph("http://localhost:7474", auth=("neo4j", "010209"))


In [20]:
def create_node(label, nodes):
    """
    label:node type
    nodes:node's name
    """
    # for node_name in nodes:
    node = Node(label, name=nodes)

    # graph.create(node)
    graph.merge(node, label, "name")
    #print('create node success! total:', len(nodes))
    #print('create node success!')


In [21]:
def create_relationship(start_node_label, end_node_label, start_node_name,
                        end_node_name, rel_type, rel_name):
    """
    example: 小明训练小华 小明和小华的类型是学生 训练的属性是1000米
    start_node_label = '学生'
    end_node_label = '学生'
    start_node_name = '小明'
    end_node_name = '小华'
    rel_type = '训练'
    rel_name = '1000米'
    """
    #     query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (
    query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' merge (p)-[rel:%s{name:'%s'}]->(q)" % (
        start_node_label, end_node_label, start_node_name, end_node_name,
        rel_type, rel_name)
    try:
        graph.run(query)
        # print('create relationship success! query:\n', query)
    except Exception as e:
        print(e)


In [22]:
import spacy
# 加载lg模型
nlp = spacy.load("zh_core_web_lg")
cfg = {"segmenter": "jieba"}
nlp.from_config({"nlp": {"tokenizer": cfg}})
# nlp.tokenizer.initialize(pkuseg_model="mixed")


In [23]:
from snownlp import summary
from snownlp import *


def load_data(title, text):
    """单篇文章导入图数据库"""
    doc = nlp(text)

    # 存放spcay识别出来的ents
    entity = set()
    location = []
    person = []
    unit = []
    for ent in doc.ents:
        #     print(ent.text, ent.label_)
        if ent.label_ in ['ORG', 'PERSON', 'WORK_OF_ART', 'LAW', 'LANGUAGE']:
            entity.add((ent.text, ent.label_))

        if ent.label_ == 'ORG':
            unit.append((ent.text, 'org'))
        if ent.label_ == 'LOC':
            location.append((ent.text, 'location'))
        if ent.label_ == 'GPE':  # 国家
            location.append((ent.text, 'location'))
        if ent.label_ == 'PERSON':
            person.append((ent.text, 'person'))

    relation = []
    # objects:存放entity和nsubj（名词性主语）
    objects = []
    times = []
    objects.extend(entity)

    # 关系抽取
    for token in doc:
        # print('{0}({1}) <-- {2} -- {3}({4})'.format(token.text, token.tag_,
        #                                             token.dep_, token.head.text,
        #                                             token.head.tag_))

        if token.dep_ == 'compound:nn' and token.tag_ == 'NT' and token.head.tag_ == 'NT':
            times.append((token.text+token.head.text, 'time'))

        # if token.dep_ == 'nsubj':
            #         print('{0}({1}) <-- {2} -- {3}({4})'.format(token.text, token.tag_,
            #                                                     token.dep_,
            #                                                     token.head.text,
            #                                                     token.head.tag_))
        #     objects.append((token.text, token.tag_))

        # if token.head.tag_ == 'VV':
        #     if token.tag_ == 'NN':
        #         if objects[-1][0] != token.text:
                    #             print('{0}({1}) <-- {2} -- {3}({4})'.format(token.text, token.tag_,
                    #                                                 token.dep_, token.head.text,
                    #                                                 token.head.tag_))
                    # relation.append(
                    #     (objects[-1][0], objects[-1][1], token.head.text,
                    #      token.head.tag_, token.text, token.tag_))

    # 创建Node Title 目前是一篇文章的root
    create_node(label='Title', nodes=title)

    # 创建Node Content 存放一篇文章的所有内容
    content_name = 'Content'+title
    create_node(label='Content', nodes=content_name)

    # 创建Node File 存放一篇文章的原始文件
    create_node(label='File', nodes=text)

    # 创建Node Time 存放一篇文章的所有时间
    time_name = 'Time'+title
    create_node(label='Time', nodes=time_name)

    # 创建Node Location 存放一篇文章的所有地理位置
    loc_name = 'Loc'+title
    create_node(label='Location', nodes=loc_name)

    # 创建Node Person 存放一篇文章的所有人
    person_name = 'Person'+title
    create_node(label='Person', nodes=person_name)

    # 创建Node Unit 存放一篇文章的所有组织ORG
    org_name = 'ORG'+title
    create_node(label='ORG', nodes=org_name)

    # 创建relation Title has_content Content
    create_relationship('Title', 'Content', title,
                        content_name, 'Content', 'has_content')

    # 创建relation Title has_file File
    create_relationship('Title', 'File', title,
                        text, 'File', 'has_file')

    # 创建relation Title has_time Time
    create_relationship('Title', 'Time', title,
                        time_name, 'Time', 'has_time')

    # 将所有子时间结点与Time（时间root）创建关系
    for i in times:
        create_node(label=i[1], nodes=i[0])
        create_relationship('Time', i[1], time_name,
                            i[0], 'time', 'has_Time')

    # 创建relation Title has_person Person
    create_relationship('Title', 'Person', title,
                        person_name, 'Person', 'has_person')

    # 将所有子结点与Person（root）创建关系
    for i in person:
        create_node(label=i[1], nodes=i[0])
        create_relationship('Person', i[1], person_name,
                            i[0], 'person', 'has_person')

    create_relationship('Title', 'Location', title,
                        loc_name, 'Location', 'has_location')

    # 将所有子结点与Location（root）创建关系
    for i in location:
        create_node(label=i[1], nodes=i[0])
        create_relationship('Location', i[1], loc_name,
                            i[0], 'locatiion', 'has_location')

    create_relationship('Title', 'ORG', title,
                        org_name, 'ORG', 'has_org')

    # 将所有子结点与ORG（root）创建关系
    for i in unit:
        create_node(label=i[1], nodes=i[0])
        create_relationship('ORG', i[1], org_name,
                            i[0], 'organization', 'has_organization')

    # 为Content结点创建子节点

    s = SnowNLP(text)
    for i in set(s.summary()):
        create_node(label='abstract', nodes=i)
        create_relationship('Content', 'abstract', content_name,
                            i, 'Content', 'has_content')
    # for i in relation:
    #     start_node_name = i[0]
    #     start_node_label = i[1]
    #     rel_name = i[2]
    #     rel_type = i[3]
    #     end_node_name = i[4]
    #     end_node_label = i[5]

    #     create_node(label=i[5], nodes=i[4])
    #     create_node(label=i[1], nodes=i[0])

    #     create_relationship(start_node_label, end_node_label, start_node_name,
    #                         end_node_name, rel_type, rel_name)

    #     create_relationship('Content', end_node_label, content_name,
    #                         end_node_name, 'Content', 'has_content')

    #     create_relationship('Content', start_node_label, content_name,
    #                         start_node_name, 'Content', 'has_content')


In [24]:
# 数据集正式导入数据库
from tqdm import tqdm

# count = 0
for i in tqdm(iterable=dataset, desc="loading"):
    try:
        load_data(title=i[0], text=i[1])
    except IndexError:
        raise Exception("运行完成或IndexError")

print("Success Loading!")
    # if count % 25 == 0:
    #     f"Success {count}/{len(dataset)}"
    # count += 1


loading: 100%|██████████| 61/61 [00:49<00:00,  1.23it/s]

Success Loading!
